# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import datetime
import os
import random as rn

import numpy as np
import tensorflow as tf
from PIL import Image

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
rn.seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
train_doc = np.random.permutation(open("/content/drive/MyDrive/Gesture Recognition using Deep Learning/Data/train.csv").readlines())
val_doc = np.random.permutation(open("/content/drive/MyDrive/Gesture Recognition using Deep Learning/Data/val.csv").readlines())
batch_size = 64

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with some of the parts of the generator function such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size, image_size, img_idx, num_classes):
    w, h = image_size

    print("Source path = ", source_path, "; batch size =", batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t) / batch_size)
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size, len(img_idx), w, h, 3))
            batch_labels = np.zeros((batch_size, num_classes))
            for folder in range(batch_size):
                imgs = os.listdir(
                    source_path + "/" + t[folder + (batch * batch_size)].split(";")[0]
                )
                for idx, item in enumerate(img_idx):
                    image = Image.open(
                        source_path
                        + "/"
                        + t[folder + (batch * batch_size)].strip().split(";")[0]
                        + "/"
                        + imgs[item]
                    )

                    if image.size[1] == 160:
                        image = image.resize(image_size)
                    else:
                        image = image.resize(image_size)

                    image = np.array(image).astype(np.float32)

                    batch_data[folder, idx, :, :, 0] = image[:, :, 0] - 104
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1] - 117
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2] - 123

                batch_labels[
                    folder, int(t[folder + (batch * batch_size)].strip().split(";")[2])
                ] = 1
            yield batch_data, batch_labels

        if (len(t) % batch_size) != 0:
            batch_data = np.zeros((len(t) % batch_size, len(img_idx), w, h, 3))
            batch_labels = np.zeros((len(t) % batch_size, num_classes))
            for folder in range(len(t) % batch_size):
                imgs = os.listdir(
                    source_path
                    + "/"
                    + t[folder + (num_batches * batch_size)].split(";")[0]
                )
                for idx, item in enumerate(img_idx):
                    image = Image.open(
                        source_path
                        + "/"
                        + t[folder + (num_batches * batch_size)].strip().split(";")[0]
                        + "/"
                        + imgs[item]
                    )
                    if image.size[1] == 160:
                        image = image.resize(image_size)
                    else:
                        image = image.resize(image_size)

                    image = np.array(image).astype(np.float32)

                    batch_data[folder, idx, :, :, 0] = image[:, :, 0] - 104
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1] - 117
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2] - 123

                batch_labels[
                    folder,
                    int(t[folder + (num_batches * batch_size)].strip().split(";")[2]),
                ] = 1

            yield batch_data, batch_labels

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = "/content/drive/MyDrive/Gesture Recognition using Deep Learning/Data/train"
val_path = "/content/drive/MyDrive/Gesture Recognition using Deep Learning/Data/val"
num_train_sequences = len(train_doc)
print("# training sequences =", num_train_sequences)
num_val_sequences = len(val_doc)
print("# validation sequences =", num_val_sequences)
num_epochs = 70
print("# epochs =", num_epochs)
image_size = (120, 120)
print("# image_size =", image_size)
num_classes = 5
print("# num_classes =", num_classes)
img_idx = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
print("# img_idx =", img_idx)

# training sequences = 663
# validation sequences = 100
# epochs = 70
# image_size = (120, 120)
# num_classes = 5
# img_idx = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]


## Model
Here you make the model using different functionalities that Keras provides. You might want to use `TimeDistributed`, `GRU` and other RNN structures after doing transfer learning. Also remember that the last layer is the softmax. Remember that the network is designed in such a way that the model is able to fit in the memory of the webcam.

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import GRU, Dense, Dropout, Flatten, TimeDistributed
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model, Sequential

w, h = image_size

base_model = VGG16(include_top=False, weights="imagenet", input_shape=(w, h, 3))
x = base_model.output
x = Flatten()(x)
# x.add(Dropout(0.5))
features = Dense(64, activation="relu")(x)
conv_model = Model(inputs=base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(TimeDistributed(conv_model, input_shape=(15, 120, 120, 3)))
model.add(GRU(32, return_sequences=True))
model.add(GRU(16))
model.add(Dropout(0.5))
model.add(Dense(8, activation="relu"))
model.add(Dense(num_classes))

58889256/58889256 [==============================] - 4s 0us/step


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
adam = optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=adam,
    loss=CategoricalCrossentropy(from_logits=True),
    metrics=["categorical_accuracy"],
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 15, 64)            15009664  
 ributed)                                                        
                                                                 
 gru (GRU)                   (None, 15, 32)            9408      
                                                                 
 gru_1 (GRU)                 (None, 16)                2400      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 5)                 45        
                                                        

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(
    train_path, train_doc, batch_size, image_size, img_idx, num_classes
)
val_generator = generator(
    val_path, val_doc, batch_size, image_size, img_idx, num_classes
)

In [ ]:
model_name = (
    "/content/drive/MyDrive/Gesture Recognition using Deep Learning/CheckPoints/"
    + "model_init_conv_lstm"
    + "_"
    + str(curr_dt_time).replace(" ", "").replace(":", "_")
    + "/"
)

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = (
    model_name
    + "model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5"
)

checkpoint = ModelCheckpoint(
    filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
)

LR = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
    mode="min",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.00001,
)
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences % batch_size) == 0:
    steps_per_epoch = int(num_train_sequences / batch_size)
else:
    steps_per_epoch = (num_train_sequences // batch_size) + 1

if (num_val_sequences % batch_size) == 0:
    validation_steps = int(num_val_sequences / batch_size)
else:
    validation_steps = (num_val_sequences // batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
    callbacks=callbacks_list,
    validation_data=val_generator,
    validation_steps=validation_steps,
    class_weight=None,
    workers=1,
    initial_epoch=0,
)

In [13]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/Gesture Recognition using Deep Learning/CheckPoints/model_init_conv_lstm_2023-09-2708_21_44.565528/model-00038-0.16660-0.97738-0.63425-0.77000.h5")
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
    callbacks=callbacks_list,
    validation_data=val_generator,
    validation_steps=validation_steps,
    class_weight=None,
    workers=1,
    initial_epoch=38,
)

Source path =  /content/drive/MyDrive/Gesture Recognition using Deep Learning/Data/train ; batch size = 64
Epoch 39/70
11/11 [==============================] - ETA: 0s - loss: 0.1652 - categorical_accuracy: 0.9774   Source path =  /content/drive/MyDrive/Gesture Recognition using Deep Learning/Data/val ; batch size = 64

Epoch 39: saving model to /content/drive/MyDrive/Gesture Recognition using Deep Learning/CheckPoints/model_init_conv_lstm_2023-09-2710_14_07.652404/model-00039-0.16525-0.97738-0.63518-0.77000.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - 8248s 821s/step - loss: 0.1652 - categorical_accuracy: 0.9774 - val_loss: 0.6352 - val_categorical_accuracy: 0.7700 - lr: 3.1250e-05
Epoch 40/70
11/11 [==============================] - ETA: 0s - loss: 0.1658 - categorical_accuracy: 0.9774
Epoch 40: saving model to /content/drive/MyDrive/Gesture Recognition using Deep Learning/CheckPoints/model_init_conv_lstm_2023-09-2710_14_07.652404/model-00040-0.16575-0.97738-0.73479-0.74000.h5
11/11 [==============================] - 53s 5s/step - loss: 0.1658 - categorical_accuracy: 0.9774 - val_loss: 0.7348 - val_categorical_accuracy: 0.7400 - lr: 3.1250e-05
Epoch 41/70
11/11 [==============================] - ETA: 0s - loss: 0.1773 - categorical_accuracy: 0.9759
Epoch 41: saving model to /content/drive/MyDrive/Gesture Recognition using Deep Learning/CheckPoints/model_init_conv_lstm_2023-09-2710_14_07.652404/model-00041-0.17733-0.97587-0.63313-0.77000.h5
11/11 [==============================] - 50s 5s/step

In [1]:
model.save_weights("model_weights.h5")

NameError: ignored